In [1]:
import re
import os
from glob import glob

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 0
MODEL_NAME  = 'gpt-4o-mini'
EMBEDDING_NAME = 'text-embedding-3-large'

COLLECTION_NAME = 'korean_history'
PERSIST_DIRECTORY= 'vector_store/korean_history_db'
name = ["고대", "고려", "근대", "조선", "현대"]

In [3]:
# Split
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name=MODEL_NAME,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

In [5]:
# db 생성
for i in name:
    file = glob(f"./pdf/인물/{i}/*.pdf")
    document_list = []
    for path in file:
        loader = PyMuPDFLoader(path)
        load_docs = loader.load()
        full_text = [doc.page_content for doc in load_docs] 
        full_text = ''.join(full_text)
        full_text = re.sub(r"관련사료", "", full_text)
        full_text = re.sub(r"\([一-龥]+\)", "", full_text)
        full_text = re.sub(r"\n", "", full_text)

        docs = splitter.split_text(full_text)
        id = "_".join(os.path.splitext(os.path.basename(path))[0].split('_')[:2])
        title = os.path.splitext(os.path.basename(path))[0].split('_')[-1]
        metadata = {
        "id":id,
        "title": title,
        "full_text": full_text
        }

        for doc in docs:
            _doc = Document(metadata=metadata, page_content=doc)
            document_list.append(_doc)

    embedding_model = OpenAIEmbeddings(model=EMBEDDING_NAME)

    vector_store = Chroma.from_documents(
        documents=document_list,
        embedding=embedding_model,
        ids=[doc.metadata['title'] for doc in document_list],
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )

    print(len(document_list))

72
85
88


c:\Users\Playdata\SKN06\단위 프로젝트\Prj03\prj03\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:322: UserWarning: Warning: Empty content on page 3 of document ./pdf/인물/조선\인물_조선_이제마.pdf
  warnings.warn(


155
6


In [4]:
# db 연결
embedding_model = OpenAIEmbeddings(model=EMBEDDING_NAME)
vector_store = Chroma(collection_name=COLLECTION_NAME, persist_directory=PERSIST_DIRECTORY, embedding_function=embedding_model)

In [5]:
vector_store._collection.count()

668

In [6]:
retriever = vector_store.as_retriever(search_type="mmr")

In [7]:
query = "이순신에 대해 알려줘"
result = retriever.invoke(query)
result

[Document(metadata={'full_text': '신익희[申翼熙]독립과 민주주의 실현을 위해 노력한 정치인1894년(고종 31) ~ 1956년 개요신익희는 1894년 경기도 광주에서 사헌부 대사헌, 한성부 판윤을 지낸 신단의 여섯 아들 중 막내로 태어났다. 일본유학을 통해 근대학문을 익힌 그는 3·1운동을 주도했고 일제의검거를 피해 중국으로 망명한 후에는 대한민국 임시정부, 민족혁명당, 조선청년전위동맹 등에참여해 독립운동을 펼쳤다. 대한민국 임시정부 내무부장으로서 해방을 맞은 그는 해방정국에서임시정부의 신탁통치 반대운동을 이끌었으나, 이후 임정과 결별하고 이승만 지지로 돌아서 남한단독정부 수립에 참여했다. 정부 수립 이후, 그는 집권 연장을 위한 이승만의 위법적 정치를 비판하며 야당 정치인의 길을 걸었다. 1956년 3대 대통령 선거 당시 민주당 대통령 후보로 출마하였으나 선거운동 도중 뇌일혈로 급사했다.1 어린 시절부터 3·1운동까지신익희는 어려서는 집안에 개설된 가내서당에서 한학을 배웠고, 1908년 15세가 되던 해 관립한성외국어학교 영어과에 진학하여 영어와 근대학문을 배웠다. 외국어학교 졸업 후에는 일본 와세다 대학 정치경제학과에 유학했다. 유학시절 그는 동경유학생들의 집합체이자 1919년 2·8 독립운동을 내용적으로 주도한 것으로 알려진 조선유학생학우회를 조직하는 데 핵심적인 역할을 담당하였으며 , 일본 유학생들뿐 아니라 국내 신지식인층에게도 큰 영향을 미친 학우회의기관지 『학지광』의 편집 겸 발행인을 지냈다. 와세다 대학 졸업 후에는 귀국하여 1917년에는 중동학교에서 교편을 잡았고 1918년에는 고려대학교의 전신인 보성법률상업학교로 옮겨3·1운동 이후 중국으로 망명할 때까지 법률담당 강사로서 비교헌법·국제공법·재정학 등을 강의했다.신익희는 1918년 미국 대통령 우드로 윌슨(Thomas Woodrow Wilson)의 민족자결주의가 국내에전해지자 일본 유학생 출신인 최남선, 최린, 송진우 등과 함께 독립운동 모의에 적극 참여하였고,이 과정

In [8]:
result = vector_store.similarity_search_with_score(
    query="이순신에 대해 알려줘",
    k=5
)
result

[(Document(metadata={'full_text': '신익희[申翼熙]독립과 민주주의 실현을 위해 노력한 정치인1894년(고종 31) ~ 1956년 개요신익희는 1894년 경기도 광주에서 사헌부 대사헌, 한성부 판윤을 지낸 신단의 여섯 아들 중 막내로 태어났다. 일본유학을 통해 근대학문을 익힌 그는 3·1운동을 주도했고 일제의검거를 피해 중국으로 망명한 후에는 대한민국 임시정부, 민족혁명당, 조선청년전위동맹 등에참여해 독립운동을 펼쳤다. 대한민국 임시정부 내무부장으로서 해방을 맞은 그는 해방정국에서임시정부의 신탁통치 반대운동을 이끌었으나, 이후 임정과 결별하고 이승만 지지로 돌아서 남한단독정부 수립에 참여했다. 정부 수립 이후, 그는 집권 연장을 위한 이승만의 위법적 정치를 비판하며 야당 정치인의 길을 걸었다. 1956년 3대 대통령 선거 당시 민주당 대통령 후보로 출마하였으나 선거운동 도중 뇌일혈로 급사했다.1 어린 시절부터 3·1운동까지신익희는 어려서는 집안에 개설된 가내서당에서 한학을 배웠고, 1908년 15세가 되던 해 관립한성외국어학교 영어과에 진학하여 영어와 근대학문을 배웠다. 외국어학교 졸업 후에는 일본 와세다 대학 정치경제학과에 유학했다. 유학시절 그는 동경유학생들의 집합체이자 1919년 2·8 독립운동을 내용적으로 주도한 것으로 알려진 조선유학생학우회를 조직하는 데 핵심적인 역할을 담당하였으며 , 일본 유학생들뿐 아니라 국내 신지식인층에게도 큰 영향을 미친 학우회의기관지 『학지광』의 편집 겸 발행인을 지냈다. 와세다 대학 졸업 후에는 귀국하여 1917년에는 중동학교에서 교편을 잡았고 1918년에는 고려대학교의 전신인 보성법률상업학교로 옮겨3·1운동 이후 중국으로 망명할 때까지 법률담당 강사로서 비교헌법·국제공법·재정학 등을 강의했다.신익희는 1918년 미국 대통령 우드로 윌슨(Thomas Woodrow Wilson)의 민족자결주의가 국내에전해지자 일본 유학생 출신인 최남선, 최린, 송진우 등과 함께 독립운동 모의에 적극 참여하였고,이 과

In [21]:
# Prompt Template 생성
messages = [
        ("ai", """
    너는 한국사에 대해서 해박한 지식을 가진 역사전문가야.
    내가 역사적 인물 또는 사건에 대해 말하면 그 인물과 사건을 이해가 쉽게, 흥미를 잃지 않게 쉬운용어로 풀어서 설명해주면 돼.

    문서에 없는 내용은 답변할 수 없습니다. 모른다고 답변 하세요.

    인물의 이름 :
    시대 :
    인물에 대해 알고 싶은 것 :
{context}"""),
        ("human", "{question}"),
    ]
prompt_template = ChatPromptTemplate(messages)

# 모델
model = ChatOpenAI(model="gpt-4o-mini")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련 문서 조회) -> prompt_template(prompt 생성) model(정답) -> output parser
chain = {"context":retriever, "question":RunnablePassthrough()} | prompt_template | model | parser

In [22]:
result = chain.invoke("트럼프에 대해 알려줘") # 나옹혜근

In [23]:
print(result)

죄송하지만, 트럼프에 대한 내용은 문서에 포함되어 있지 않습니다. 다른 역사적 인물이나 사건에 대해 질문해 주시면 답변해 드리겠습니다.
